In [2]:
import osmnx as ox
import time
from shapely.geometry import Polygon
import os

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = gdf_edges.index
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)

osmnx version 2.0.1


### Download network in a boundingbox

In [4]:
import osmnx as ox
import time
from shapely.geometry import Polygon

# Define bounding box coordinates (correcting x1, x2, y1, y2)
bounds = (40.09032722446688, 116.10590363281044, 39.771482256675114, 116.66101710094082)
y1, x1, y2, x2 = bounds  # Reordering to (lat1, lon1, lat2, lon2)

# Create a polygon from bounding box
boundary_polygon = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])

# Get the drive network
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')

# Save graph as a shapefile
start_time = time.time()
ox.io.save_graph_shapefile(G, filepath='./beijing2')  # Updated function call
print("--- %s seconds ---" % (time.time() - start_time))


AttributeError: module 'osmnx.io' has no attribute 'save_graph_shapefile'

### Download network in a city

In [ ]:
place ="Stockholm, Sweden"
G = ox.graph_from_place(place, network_type='all', which_result=2)
save_graph_shapefile_directional(G, filepath='stockholm')

### Download network with a boundary

In [ ]:
# Download by a boundary polygon in geojson
import osmnx as ox
from shapely.geometry import shape
json_file = open("stockholm_boundary.geojson")
import json
data = json.load(json_file)
boundary_polygon = shape(data["features"][0]['geometry'])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
save_graph_shapefile_directional(G, filepath='stockholm')

### Fix for Beijing issues

In [ ]:
import osmnx as ox
import geopandas as gpd
import time
import os

# Define place name (center of Beijing)
place_name = "Tiananmen, Beijing, China"

# Download a drive network within a specified radius (e.g., 15km)
G = ox.graph_from_address(place_name, dist=15000, network_type="all")

# Convert the graph to GeoDataFrames
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

# Define output directory
output_dir = "./beijing"
os.makedirs(output_dir, exist_ok=True)

# Save nodes and edges as ESRI Shapefiles
start_time = time.time()
gdf_nodes.to_file(f"{output_dir}/nodes.shp", encoding="utf-8")
gdf_edges.to_file(f"{output_dir}/edges.shp", encoding="utf-8")
print(f"Graph saved in {output_dir}")
print("--- %s seconds ---" % (time.time() - start_time))


/tmp/ipykernel_6769/2385536551.py:21: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_nodes.to_file(f"{output_dir}/nodes.shp", encoding="utf-8")
/home/flo/thesis-1/.venv/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'street_count' to 'street_cou'
  ogr_write(


Graph saved in ./beijing
--- 13.868499755859375 seconds ---


add missing fid column


In [6]:
import geopandas as gpd

# Load the edges shapefile
edges = gpd.read_file("../osmnx_example/beijing/edges.shp")

# Add a unique 'fid' column
edges["fid"] = edges.index  # Unique index as ID

# Save the fixed shapefile
edges.to_file("../osmnx_example/beijing/edges.shp", encoding="utf-8")

print("Fixed shapefile saved.")

Fixed shapefile saved.


In [7]:
edges = gpd.read_file("../osmnx_example/beijing/edges.shp")
print(edges.columns)


Index(['u', 'v', 'key', 'osmid', 'access', 'highway', 'oneway', 'reversed',
       'length', 'name', 'lanes', 'bridge', 'maxspeed', 'ref', 'service',
       'tunnel', 'junction', 'width', 'est_width', 'landuse', 'fid',
       'geometry'],
      dtype='object')
